In [6]:
import os
import struct
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import scipy.ndimage

# Set Plotly renderer (optional when saving images)
pio.renderers.default = "browser"

# File paths
file_paths = {
    "v1": "examples/v1/Dime.tmd",
    "v2": "examples/v2/Dime.tmd",
    "gelsight": "examples/gelsight/circle_0mm_100g_heightmap_linear_detrend.tmd",
}

def check_version(file_path, num_bytes=64):
    """Determines the file version based on header content."""
    with open(file_path, 'rb') as f:
        header_bytes = f.read(num_bytes)
    return 2 if "v2" in header_bytes.decode('ascii', errors='replace') else 1

def process_file(file_path, force_offset=None):
    """
    Processes a TMD file while handling variations in header length.
    
    Reads metadata and height map data from the file.
    """
    if not os.path.exists(file_path):
        return None, None

    version = check_version(file_path)
    with open(file_path, 'rb') as f:
        if version == 1:
            f.seek(28)  # Offset for v1 files
            print("⚠️ Detected v1 file format. Reading metadata...")
        elif version == 2:
            f.seek(32)  # Default for v2 files
            print("⚠️ Detected v2 file format. Reading metadata...")
            try:
                comment = f.read(24).decode('ascii').strip()
                print(f"Comment: {comment}")
            except Exception as e:
                comment = None
                f.seek(33)  # Reset to read the rest of the data
        else:
            return None, None

        # Read dimensions and offsets
        width = struct.unpack('<I', f.read(4))[0]
        print(f"Width: {width}")
        height = struct.unpack('<I', f.read(4))[0]
        print(f"Height: {height}")
        x_length = struct.unpack('<f', f.read(4))[0]
        print(f"X Length: {x_length}")
        y_length = struct.unpack('<f', f.read(4))[0]
        print(f"Y Length: {y_length}")
        x_offset = struct.unpack('<f', f.read(4))[0]
        print(f"X Offset: {x_offset}")
        y_offset = struct.unpack('<f', f.read(4))[0]
        print(f"Y Offset: {y_offset}")

        # Apply forced offsets if provided
        if force_offset:
            x_offset, y_offset = force_offset
            print(f"⚠️ Using forced offsets: x_offset={x_offset}, y_offset={y_offset}")

        # Read remaining data as the height map and adjust if size is off
        height_map_data = np.frombuffer(f.read(), dtype=np.float32)
        expected_size = width * height

        if height_map_data.size < expected_size:
            print(f"Warning: Incomplete height map. Expected {expected_size}, got {height_map_data.size}.")
            height_map = np.pad(height_map_data, (0, expected_size - height_map_data.size), mode='constant')
        elif height_map_data.size > expected_size:
            print(f"Warning: Extra data detected. Expected {expected_size}, got {height_map_data.size}. Trimming excess.")
            height_map = height_map_data[:expected_size]
        else:
            height_map = height_map_data

        height_map = height_map.reshape((height, width))

    metadata = {
        "version": version,
        "width": width,
        "height": height,
        "x_length": x_length,
        "y_length": y_length,
        "x_offset": x_offset,
        "y_offset": y_offset,
        "comment": comment if version == 2 else None,
    }
    return metadata, height_map

def plot_height_map(height_map, title="Height Map", filename="height_map.html"):
    """
    Plots the height map as a 3D surface plot using Plotly.
    """
    fig = go.Figure(go.Surface(z=height_map))
    fig.update_layout(title=title)
    fig.write_html(filename)
    print(f"Saved height map plot as {filename}")
    return filename
    

def compute_normal_map(height_map, x_length, y_length):
    """
    Computes the normal map from the height map.
    
    Uses the physical dimensions to calculate spatial gradients and then
    computes normalized surface normals.
    """
    height, width = height_map.shape
    # Compute spacing along x and y axes
    dx = x_length / (width - 1)
    dy = y_length / (height - 1)
    
    # Compute gradients. np.gradient returns derivatives in order (dy, dx)
    dh_dy, dh_dx = np.gradient(height_map, dy, dx)
    
    # Compute normal vector components: N = (-dh/dx, -dh/dy, 1)
    nx = -dh_dx
    ny = -dh_dy
    nz = np.ones_like(height_map)
    
    # Normalize the normal vectors
    norm = np.sqrt(nx**2 + ny**2 + nz**2)
    normal_map = np.stack((nx/norm, ny/norm, nz/norm), axis=2)
    
    return normal_map

def normal_map_to_image(normal_map):
    """
    Converts a normal map (with components in [-1, 1]) to an 8-bit RGB image.
    
    Maps the normal components from [-1, 1] to [0, 255].
    """
    img = ((normal_map + 1) / 2 * 255).astype(np.uint8)
    return img

def save_normal_map_image(img, title="Normal Map", filename="normal_map.png"):
    """
    Saves the normal map image as a PNG file using Plotly.
    """
    fig = go.Figure(go.Image(z=img))
    fig.update_layout(title=title)
    fig.write_image(filename)  # Requires kaleido for image export
    print(f"Saved normal map image as {filename}")
    return filename

def crop_height_map_z_threshold(height_map, z_min, z_max):
    """
    Crops the height map based on a z threshold.
    
    Values below z_min are set to z_min, and values above z_max are set to z_max.
    Returns the cropped height map.
    """
    cropped_map = np.clip(height_map, z_min, z_max)
    return cropped_map

# Set forced offsets (None means use extracted values)
FORCE_OFFSET = None  # Example: (0.22, 0.22) to override

# Process the files with optional forced offsets
metadata_v1, height_map_v1 = process_file(file_paths["v1"], force_offset=FORCE_OFFSET)
metadata_v2, height_map_v2 = process_file(file_paths["v2"], force_offset=FORCE_OFFSET)
metadata_gelsight, height_map_gelsight = process_file(file_paths["gelsight"], force_offset=FORCE_OFFSET)

def apply_gaussian_filter(height_map, sigma=2):
    """
    Applies a Gaussian filter to the height map using the specified sigma.
    
    Returns the smoothed height map.
    """
    smoothed = scipy.ndimage.gaussian_filter(height_map, sigma=sigma)
    return smoothed

# Compute and save normal maps as PNG images if the height map is successfully read
# if height_map_v1 is not None:
#     normal_v1 = compute_normal_map(height_map_v1, metadata_v1["x_length"], metadata_v1["y_length"])
#     img_v1 = normal_map_to_image(normal_v1)
#     save_normal_map_image(img_v1, title="Normal Map (v1)", filename="normal_map_v1.png")

# if height_map_v2 is not None:
#     normal_v2 = compute_normal_map(height_map_v2, metadata_v2["x_length"], metadata_v2["y_length"])
#     img_v2 = normal_map_to_image(normal_v2)
#     save_normal_map_image(img_v2, title="Normal Map (v2)", filename="normal_map_v2.png")

# if height_map_gelsight is not None:
#     normal_gelsight = compute_normal_map(height_map_gelsight, metadata_gelsight["x_length"], metadata_gelsight["y_length"])
#     img_gelsight = normal_map_to_image(normal_gelsight)
#     save_normal_map_image(img_gelsight, title="Normal Map (GelSight)", filename="normal_map_gelsight.png")

# # Plot height maps as 3D surface plots
# if height_map_v1 is not None:
#     plot_height_map(height_map_v1, title="Height Map (v1)", filename="height_map_v1.html")

# if height_map_v2 is not None:
#     plot_height_map(height_map_v2, title="Height Map (v2)", filename="height_map_v2.html")

# if height_map_gelsight is not None:
#     plot_height_map(height_map_gelsight, title="Height Map (GelSight)", filename="height_map_gelsight.html")

if height_map_v1 is not None:
    z_min, z_max = 0.1, 0.2  # Set your desired threshold values for v1
    cropped_v1 = crop_height_map_z_threshold(height_map_v1, z_min, z_max)
    plot_height_map(cropped_v1, title="Cropped Height Map (v1)", filename="height_map_cropped_v1.html")

if height_map_v2 is not None:
    z_min, z_max = 0.1, 0.2  # Set your desired threshold values for v2
    cropped_v2 = crop_height_map_z_threshold(height_map_v2, z_min, z_max)
    plot_height_map(cropped_v2, title="Cropped Height Map (v2)", filename="height_map_cropped_v2.html")

if height_map_gelsight is not None:
    z_min, z_max = 0.2, 0.3  # Set your desired threshold values for gelsight
    cropped_gelsight = crop_height_map_z_threshold(height_map_gelsight, z_min, z_max)
    plot_height_map(cropped_gelsight, title="Cropped Height Map (GelSight)", filename="height_map_cropped_gelsight.html")

# Apply Gaussian smoothing to the height maps and plot the smoothed versions
if height_map_v1 is not None:
    smoothed_v1 = apply_gaussian_filter(height_map_v1, sigma=2)
    plot_height_map(smoothed_v1, title="Smoothed Height Map (v1)", filename="height_map_smoothed_v1.html")

if height_map_v2 is not None:
    smoothed_v2 = apply_gaussian_filter(height_map_v2, sigma=2)
    plot_height_map(smoothed_v2, title="Smoothed Height Map (v2)", filename="height_map_smoothed_v2.html")

if height_map_gelsight is not None:
    smoothed_gelsight = apply_gaussian_filter(height_map_gelsight, sigma=2)
    plot_height_map(smoothed_gelsight, title="Smoothed Height Map (GelSight)", filename="height_map_smoothed_gelsight.html")

⚠️ Detected v1 file format. Reading metadata...
Width: 300
Height: 300
X Length: 18.956600189208984
Y Length: 18.956600189208984
X Offset: 0.22351792454719543
Y Offset: 0.2231409251689911
⚠️ Detected v2 file format. Reading metadata...
Comment: Created by TrueMap v6
 
Width: 300
Height: 300
X Length: 18.956600189208984
Y Length: 18.956600189208984
X Offset: 0.0
Y Offset: 0.0
⚠️ Detected v2 file format. Reading metadata...
Width: 6024
Height: 4022
X Length: 7.675057888031006
Y Length: 5.124349594116211
X Offset: 0.0
Y Offset: 0.0
Saved height map plot as height_map_cropped_v1.html
Saved height map plot as height_map_cropped_v2.html
Saved height map plot as height_map_cropped_gelsight.html
Saved height map plot as height_map_smoothed_v1.html
Saved height map plot as height_map_smoothed_v2.html
Saved height map plot as height_map_smoothed_gelsight.html


In [7]:
import os
import struct
import numpy as np
import pickle
import lzma

# --- File Processing Function ---
def process_file(file_path, force_offset=None):
    """
    Processes a TMD file to read metadata and the height map.
    
    Returns:
        metadata: A dictionary containing file metadata.
        height_map: A 2D numpy array of height values.
    """
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None, None

    with open(file_path, 'rb') as f:
        # Determine version from header bytes
        header_bytes = f.read(64)
        version = 2 if "v2" in header_bytes.decode('ascii', errors='replace') else 1
        
        if version == 1:
            f.seek(28)  # v1 header offset
            print("Detected v1 file format. Reading metadata...")
            comment = None
        elif version == 2:
            f.seek(32)  # v2 header offset
            print("Detected v2 file format. Reading metadata...")
            try:
                comment = f.read(24).decode('ascii').strip()
            except Exception:
                comment = None
                f.seek(33)
        else:
            return None, None

        # Read dimensions and offsets
        width = struct.unpack('<I', f.read(4))[0]
        height = struct.unpack('<I', f.read(4))[0]
        x_length = struct.unpack('<f', f.read(4))[0]
        y_length = struct.unpack('<f', f.read(4))[0]
        x_offset = struct.unpack('<f', f.read(4))[0]
        y_offset = struct.unpack('<f', f.read(4))[0]

        print(f"Width: {width}, Height: {height}")
        print(f"X Length: {x_length}, Y Length: {y_length}")
        print(f"X Offset: {x_offset}, Y Offset: {y_offset}")

        if force_offset:
            x_offset, y_offset = force_offset
            print(f"Using forced offsets: x_offset={x_offset}, y_offset={y_offset}")

        # Read remaining data as the height map
        height_map_data = np.frombuffer(f.read(), dtype=np.float32)
        expected_size = width * height
        if height_map_data.size < expected_size:
            print(f"Warning: Incomplete height map. Expected {expected_size}, got {height_map_data.size}.")
            height_map = np.pad(height_map_data, (0, expected_size - height_map_data.size), mode='constant')
        elif height_map_data.size > expected_size:
            print(f"Warning: Extra data detected. Expected {expected_size}, got {height_map_data.size}. Trimming excess.")
            height_map = height_map_data[:expected_size]
        else:
            height_map = height_map_data

        height_map = height_map.reshape((height, width))

    metadata = {
        "version": version,
        "width": width,
        "height": height,
        "x_length": x_length,
        "y_length": y_length,
        "x_offset": x_offset,
        "y_offset": y_offset,
        "comment": comment if version == 2 else None,
    }
    return metadata, height_map

# --- Save with Improved Lossless Compression using LZMA ---
def save_heightmap_lzma(metadata, height_map, filename="height_map_compressed.lzma"):
    """
    Saves the metadata and height map using pickle + LZMA compression.
    
    LZMA typically gives a better compression ratio than zlib (used by np.savez_compressed).
    """
    data = {"metadata": metadata, "height_map": height_map}
    # Use the highest compression preset (9)
    compressed = lzma.compress(pickle.dumps(data, protocol=pickle.HIGHEST_PROTOCOL), preset=9)
    with open(filename, "wb") as f:
        f.write(compressed)
    print(f"Saved height map using LZMA compression to {filename}")
    return filename

# --- Load from LZMA-compressed File ---
def load_heightmap_lzma(filename="height_map_compressed.lzma"):
    """
    Loads the metadata and height map from the LZMA compressed file.
    """
    with open(filename, "rb") as f:
        compressed = f.read()
    data = pickle.loads(lzma.decompress(compressed))
    metadata = data["metadata"]
    height_map = data["height_map"]
    print(f"Loaded height map from {filename} with shape {height_map.shape}")
    return metadata, height_map

# --- Main Script ---
file_path = "examples/gelsight/circle_0mm_100g_heightmap_linear_detrend.tmd"  # Update path if needed
metadata, height_map = process_file(file_path)

if height_map is not None:
    # Save the height map with improved lossless compression using LZMA
    save_filename = save_heightmap_lzma(metadata, height_map, filename="height_map_compressed.lzma")
    
    # Load the data back to verify
    metadata_loaded, height_map_loaded = load_heightmap_lzma(save_filename)
else:
    print("Failed to load the height map.")


Detected v2 file format. Reading metadata...
Width: 6024, Height: 4022
X Length: 7.675057888031006, Y Length: 5.124349594116211
X Offset: 0.0, Y Offset: 0.0
Saved height map using LZMA compression to height_map_compressed.lzma
Loaded height map from height_map_compressed.lzma with shape (4022, 6024)
